In [ ]:
# This cell is added by sphinx-gallery
# It can be customized to whatever you like
%matplotlib inline

Quantum natural SPSA optimizer
==============================

::: {.meta}
:property=\"og:description\": Introduction to the Quantum natural SPSA
optimizer, which reduces the number of quantum measurements in the
optimization. :property=\"og:image\":
<https://pennylane.ai/qml/_images/qnspsa_cover.png>
:::

::: {.related}
tutorial\_spsa Simultaneous perturbation stochastic approximation (SPSA)
optimizer
:::

*Author: Yiheng Duan --- Posted: 18 July 2022. Last updated: 05
September 2022.*

In this tutorial, we show how we can implement the [quantum natural
simultaneous perturbation stochastic approximation (QN-SPSA)
optimizer](https://quantum-journal.org/papers/q-2021-10-20-567/) from
Gacon et al. using PennyLane.

Variational quantum algorithms (VQAs) are in close analogy to their
counterparts in classical machine learning. They both build a closed
optimization loop and utilize an optimizer to iterate on the parameters.
However, out-of-the-box classical gradient-based optimizers, such as
gradient descent, are often unsatisfying for VQAs, as quantum
measurements are notoriously expensive and gradient measurements for
quantum circuits scale poorly with the system size.

In, Gacon et al. propose QN-SPSA, which is tailored for quantum
algorithms. In each optimization step, QN-SPSA executes only 2 quantum
circuits to estimate the gradient, and another 4 for the Fubini-Study
metric tensor, independent of the problem size. This preferred scaling
makes it a promising candidate for optimization tasks for noisy
intermediate-scale quantum (NISQ) devices.


Introduction
============

In quantum machine learning (QML) and variational quantum algorithms
(VQA), an optimizer does the following two tasks:

-   It estimates the gradient of the cost function or other relevant
    metrics at the current step.
-   Based on the metrics, it decides the parameters for the next
    iteration to reduce the cost.

A simple example of such an optimizer is the vanilla gradient descent
(GD), whose update rule is written as:

$$\mathbf{x}^{(t + 1)} = \mathbf{x}^{(t)} - \eta \nabla f(\mathbf{x}^{(t)}) \label{eq:vanilla}\tag{1},$$

where $f(\mathbf{x})$ is the loss function with input parameter
$\mathbf{x}$, while $\eta$ is the learning rate. The superscript $t$
stands for the $t$-th iteration step in the optimization. Here the
gradient $\nabla f$ is estimated dimension by dimension, requiring
$O(d)$ quantum measurements ($d$ being the dimension of the parameter
space). As quantum measurements are expensive, this scaling makes GD
impractical for complicated high-dimensional circuits.

To address this unsatisfying scaling, the `simultaneous perturbation
stochastic approximation (SPSA) optimizer </demos/tutorial_spsa>`{.interpreted-text
role="doc"} replaces this dimensionwise gradient estimation with a
stochastic one. In SPSA, a random direction
$\mathbf{h} \in \mathcal{U}(\{-1, 1\}^d)$ in the parameter space is
sampled, where $\mathcal{U}(\{-1, 1\}^d)$ is a $d$-dimensional discrete
uniform distribution. The gradient component along this sampled
direction is then measured with a finite difference approach, with a
perturbation step size $\epsilon$:

$$|{\nabla}_{\mathbf{h}}f(\mathbf{x})| \equiv
\mathbf{h}\cdot {\nabla}f(\mathbf{x}) \simeq \frac{1}{2\epsilon}\big(f(\mathbf{x} + \epsilon \mathbf{h}) - f(\mathbf{x} - \epsilon \mathbf{h})\big)\label{eq:finite_diff}\tag{2}.$$

A stochastic gradient estimator
$\widehat{\boldsymbol{\nabla}}f(\mathbf{x}, \mathbf{h})_{SPSA}$ is then
constructed:

$$\widehat{\nabla f}(\mathbf{x}, \mathbf{h})_{SPSA} = | {\nabla}_{\mathbf{h}}f(\mathbf{x})|\mathbf{h}\label{eq:spsaGrad}\tag{3}.$$

With the estimator, SPSA gives the following update rule:

$$\mathbf{x}^{(t + 1)} = \mathbf{x}^{(t)} - \eta \widehat{\nabla f}(\mathbf{x}^{(t)}, \mathbf{h}^{(t)})_{SPSA} \label{eq:spsa}\tag{4},$$

where $\mathbf{h}^{(t)}$ is sampled at each step. Although this
stochastic approach cannot provide a stepwise unbiased gradient
estimation, SPSA is proved to be especially effective when accumulated
over multiple optimization steps.

On the other hand, `quantum natural gradient descent (QNG)
</demos/tutorial_quantum_natural_gradient>`{.interpreted-text
role="doc"} is a variant of gradient descent. It introduces the
Fubini-Study metric tensor $\boldsymbol{g}$ into the optimization to
account for the structure of the non-Euclidean parameter space. The
$d \times d$ metric tensor is defined as

$$\boldsymbol{g}_{ij}(\mathbf{x}) = -\frac{1}{2} \frac{\partial}{\partial \mathbf{x}_i} \frac{\partial}{\partial \mathbf{x}_j} F(\mathbf{x}', \mathbf{x})\biggr\rvert_{\mathbf{x}'=\mathbf{x}},\label{eq:fsTensor}\tag{5}$$

where
$F(\mathbf{x}', \mathbf{x}) = \bigr\rvert\langle \phi(\mathbf{x}') | \phi(\mathbf{x}) \rangle \bigr\rvert ^ 2$,
and $\phi(\mathbf{x})$ is the parameterized ansatz with input
$\mathbf{x}$. With the metric tensor, the update rule is rewritten as:

$$\mathbf{x}^{(t + 1)} = \mathbf{x}^{(t)} - \eta \boldsymbol{g}^{-1}(\mathbf{x}^{(t)}) \nabla f(\mathbf{x}^{(t)}) \label{eq:qn}\tag{6}.$$

While the introduction of the metric tensor helps to find better minima
and allows for faster convergence, the algorithm is not as scalable due
to the number of measurements required to estimate $\boldsymbol{g}$.

QN-SPSA manages to combine the merits of QNG and SPSA by estimating both
the gradient and the metric tensor stochastically. The gradient is
estimated in the same fashion as the SPSA algorithm, while the
Fubini-Study metric is computed by a second-order process with another
two stochastic perturbations:

$$\widehat{\boldsymbol{g}}(\mathbf{x}, \mathbf{h}_1, \mathbf{h}_2)_{SPSA} = \frac{\delta F }{8 \epsilon^2}\Big(\mathbf{h}_1 \mathbf{h}_2^\intercal + \mathbf{h}_2 \mathbf{h}_1^\intercal\Big) \label{eq:fs_qnspsa}\tag{7},$$

where

$$\delta F = F(\mathbf{x, \mathbf{x} + \epsilon \mathbf{h}_1} + \epsilon \mathbf{h}_2) - F (\mathbf{x, \mathbf{x} + \epsilon \mathbf{h}_1}) - F(\mathbf{x, \mathbf{x} - \epsilon \mathbf{h}_1} + \epsilon \mathbf{h}_2) + F(\mathbf{x, \mathbf{x} - \epsilon \mathbf{h}_1})\label{eq:deltaf}\tag{8},$$

and $\mathbf{h}_1, \mathbf{h}_2 \in \mathcal{U}(\{-1, 1\}^d)$ are two
randomly sampled directions.

With equation (7), QN-SPSA provides the update rule

$$\mathbf{x}^{(t + 1)} = \mathbf{x}^{(t)} - \eta \widehat{\boldsymbol{g}}^{-1}(\mathbf{x}^{(t)}, \mathbf{h}_1^{(t)}, \mathbf{h}_2^{(t)})_{SPSA} \widehat{\nabla f}(\mathbf{x}^{(t)}, \mathbf{h}^{(t)})_{SPSA} \label{eq:qnspsa}\tag{9}.$$

In each optimization step $t$, one will need to randomly sample 3
perturbation directions
$\mathbf{h}^{(t)}, \mathbf{h}_1^{(t)}, \mathbf{h}_2^{(t)}$. Equation (9)
is then applied to compute the parameters for the $(t + 1)$-th step
accordingly. This $O(1)$ update rule fits into NISQ devices well.

Numerical stability
===================

The QN-SPSA update rule given in equation (9) is highly stochastic and
may not behave well numerically. In practice, a few tricks are applied
to ensure the method's numerical stability:

Averaging on the Fubini-Study metric tensor

:   A running average is taken on the metric tensor estimated from
    equation (7) at each step $t$:

    $$\bar{\boldsymbol{g}}^{(t)}(\mathbf{x}) = \frac{1}{t + 1} \Big(\sum_{i=1}^{t}\widehat{\boldsymbol{g}}(\mathbf{x}, \mathbf{h}_1^{(i)}, \mathbf{h}_2^{(i)})_{SPSA} + \boldsymbol{g}^{(0)}\Big)\label{eq:tensorRunningAvg}\tag{10} ,$$

    where the initial guess $\boldsymbol{g}^{(0)}$ is set to be the
    identity matrix.

Fubini-Study metric tensor regularization

:   To ensure the positive semidefiniteness of the metric tensor near a
    minimum, the running average in equation (10) is regularized:

    $$\bar{\boldsymbol{g}}^{(t)}_{reg}(\mathbf{x}) = \sqrt{\bar{\boldsymbol{g}}^{(t)}(\mathbf{x}) \bar{\boldsymbol{g}}^{(t)}(\mathbf{x})} + \beta \mathbb{I}\label{eq:tensor_reg}\tag{11},$$

    where $\beta$ is the regularization coefficient. We can consider
    $\beta$ as a hyperparameter and choose a suitable value by trial and
    error. If $\beta$ is too small, it cannot protect the positive
    semidefiniteness of $\bar{\boldsymbol{g}}_{reg}$. If $\beta$ is too
    large, it will wipe out the information from the Fubini-Study metric
    tensor, reducing QN-SPSA to the first order SPSA.

    With equation (11), the QN-SPSA update rule we implement in code
    reads

    $$\mathbf{x}^{(t + 1)} = \mathbf{x}^{(t)} - \eta (\bar{\boldsymbol{g}}^{(t)}_{reg})^{-1}(\mathbf{x}^{(t)}) \widehat{\nabla f}(\mathbf{x}^{(t)}, \mathbf{h}^{(t)})_{SPSA} \label{eq:qnspsa_reg}\tag{12}.$$

Blocking condition on the parameter update

:   A blocking condition is applied onto the parameter update. The
    optimizer only accepts updates that lead to a loss value no larger
    than the one before update, plus a tolerance. Reference suggests
    choosing a tolerance that is twice the standard deviation of the
    loss.

Implementation
==============

We are now going to implement the QN-SPSA optimizer with all the tricks
for numerical stability included, and test it with a toy optimization
problem.

Let's first set up the toy example to optimize. We use a [QAOA max cut
problem](https://pennylane.readthedocs.io/en/stable/code/api/pennylane.qaoa.cost.maxcut.html)
as our testing ground.


In [ ]:
# initialize a graph for the max cut problem












# define device to be the PennyLane lightning local simulator


















# define ansatz and loss function

Let's confirm this circuit works. We generate a set of random parameters
as input, and check if the QNode `cost_function` for the circuit can be
executed.


In [ ]:
# initialize a random parameter tensor with shape (2, depth), scaled
# to [-pi, pi)

::: {.rst-class}
sphx-glr-script-out

``` {.none}
Input parameter shape: (2, 2)
Loss value: -1.5099999999999998
```
:::

With the problem set up, we will for now focus on implementing a
single-step update of the QN-SPSA. Given the current parameters
`params_curr`, we would like to compute the parameters for the next step
`params_next`. We first define a few necessary hyperparameters and
global variables.


In [ ]:
# step index


# random seed for sampling the perturbation directions



# perturbation size for the finite difference calculation


# regularization coefficient for the metric tensor


# learning rate


# initialize the metric tensor to be an identity matrix

As both the gradient estimator and the metric tensor estimator involve
getting random perturbation directions, we first implement a sampling
function that we call `get_perturbation_direction`. The function takes
the input parameter to the circuit ansatz, and returns a direction
tensor of the same shape. The direction tensor is sampled from a
discrete uniform distribution $\mathcal{U}(\{-1, 1\}^d)$ using
`random.choices`.


::: {.rst-class}
sphx-glr-script-out

``` {.none}
[[-1  1]
[ 1 -1]]
```
:::

With this function at our disposal, we implement the gradient estimator
`get_grad` following equation (2):


::: {.rst-class}
sphx-glr-script-out

``` {.none}
Estimated SPSA gradient:
[[-3.05 -3.05]
[ 3.05  3.05]]
```
:::

To estimate the raw stochastic metric tensor
$\widehat{\boldsymbol{g}}(\mathbf{x}, \mathbf{h}_1, \mathbf{h}_2)_{SPSA}$
from equation (7), we will first need to measure the state overlap
$F(\mathbf{x}_1, \mathbf{x}_2) = \bigr\rvert\langle \phi(\mathbf{x}_1) | \phi(\mathbf{x}_2) \rangle \bigr\rvert ^ 2$.
We denote the unitary transformation forming the ansatz with $U$; that
is, $\rvert\phi(\mathbf{x})\rangle = U(\mathbf{x}) \rvert0\rangle$.
Applying the adjoint operation $U^{\dagger}(\mathbf{x}_2)$ on to the
ansatz state $\rvert\phi(\mathbf{x}_1)\rangle$ followed with a
measurement in the computational basis then does the trick. The state
overlap equals the probability of a $\rvert00...0\rangle$ measurement
outcome. Note that this circuit measuring the state overlap doubles the
circuit depth of the ansatz, and therefore has longer execution time and
experiences more accumulated noise from the device. The function
`get_state_overlap` returns a state overlap value between 0 (minimum
overlap) and 1 (perfect overlap).


Let's do a quick sanity check on the state overlap calculation. From the
following cell, we can see that the overlap of a state with itself is 1,
while the number for two states from random inputs can vary between 0
and 1. This means `get_state_overlap` function works!


::: {.rst-class}
sphx-glr-script-out

``` {.none}
Perfect overlap:  1.0
Random state overlap:  0.599
```
:::

Now that we have confirmed our implementation of the state overlap, we
can proceed to compute the raw stochastic metric tensor
$\widehat{\boldsymbol{g}}(\mathbf{x}, \mathbf{h}_1, \mathbf{h}_2)_{SPSA}$.
With the function `get_raw_tensor_metric`, we sample two perturbations
with `get_perturbation_direction` independently and estimate the raw
metric tensor with equations (8) and (7).


::: {.rst-class}
sphx-glr-script-out

``` {.none}
Raw estimated metric tensor:
[[ 2.5  0.  -2.5  2.5]
[ 0.  -2.5  0.   0. ]
[-2.5  0.   2.5 -2.5]
[ 2.5  0.  -2.5  2.5]]
```
:::

Now, let\'s apply the running average in equation (10), and the
regularization in equation (11):


In [ ]:
# update metric tensor



# update step index

::: {.rst-class}
sphx-glr-script-out

``` {.none}
Updated metric tensor after the step:
[[ 1.74925075  0.         -1.24875125  1.24875125]
[ 0.          0.75024975  0.          0.        ]
[-1.24875125  0.          1.74925075 -1.24875125]
[ 1.24875125  0.         -1.24875125  1.74925075]]
```
:::

Equation (12) requires computing the inverse of the metric tensor. A
numerically more stable approach is to solve the equivalent linear
equation for $\mathbf{x}^{(t + 1)}$:

$$\bar{\boldsymbol{g}}^{(t)}_{reg}(\mathbf{x}^{(t)})\big( \mathbf{x}^{(t)} - \mathbf{x}^{(t + 1)}\big) =  \eta  \widehat{\nabla f}(\mathbf{x}^{(t)}, \mathbf{h}^{(t)})_{SPSA} \label{eq:lin_solver}\tag{13}.$$


::: {.rst-class}
sphx-glr-script-out

``` {.none}
Next parameters:
[[-1.03117138 -0.54824992]
[-2.03318839  0.80331292]]
```
:::

Now, it is the time to apply the blocking condition. Let's first try the
proposal in to use twice the sample standard deviation of the loss at
the current step as the tolerance. To collect such a sample, we need to
repeat the QNode execution for the loss `cost_function` for, say, 10
times. The straightforward implementation goes as follows:


In [ ]:
# block updates that lead to significant increase
# of the loss value

::: {.rst-class}
sphx-glr-script-out

``` {.none}
Next parameters after blocking:
[[-1.03117138 -0.54824992]
[-2.03318839  0.80331292]]
```
:::

As quantum measurements are generally expensive, computing the tolerance
this way adds significant overhead to the QN-SPSA optimizer. To be
specific, in each step of the optimization, QN-SPSA only requires
executing 2 circuits for the gradient, and 4 for the metric tensor. Yet
in the approach above, there are an additional 10 (from the repeat
number) + 1 circuits required to apply the blocking.

To address this issue, we propose to define the tolerance as the
standard deviation of the loss values of the past $N$ steps instead,
where $N$ is a hyperparameter we choose. The intuition here is that when
the optimizer is working in a fast-descending regime, the blocking
condition is unlikely to be triggered, as new loss values are often
smaller than the previous ones. On the other hand, when the optimizer is
working in a rather flat energy landscape, losses from the past $N$
steps could be very similar to the current loss value. In this regime,
the tolerance defined from both approaches should be close.

The implementation of this new tolerance is shown below:


In [ ]:
# define number of steps to track

# track the past losses in an array



# stepwise update



# k has been updated above

The efficacy of this new tolerance definition is confirmed by
reproducing the experiment on QN-SPSA in Fig. 1(b) from reference. In
the following figure, we show the performance of the optimizer with the
two tolerance definitions for an 11-qubit system. The shaded areas are
the profiles of 25 trials of the experiment. One can confirm the
past-$N$-step ($N=5$ for the plot) standard deviation works just as
well. With the new choice of the tolerance, for each step, the QN-SPSA
will only need to execute 2 (for gradient) + 4 (for metric tensor) + 2
(for the current and the next-step loss) = 8 circuits. In practice, we
measure a 50% reduction in the stepwise optimization time.

The test is done with [Amazon Braket Hybrid
Jobs](https://docs.aws.amazon.com/braket/latest/developerguide/braket-jobs.html),
as it is a handy tool to scale up experiments systematically. We will
show how to do that towards the end of the tutorial.

![](../demonstrations/qnspsa/qnspsa_new_tol.png){.align-center
width="80.0%"}

Similarly, with Hybrid Jobs, we can confirm that blocking is necessary
for this second-order SPSA optimizer, though it does not make much
difference for SPSA. Here, the envelope of the QN-SPSA curves without
blocking is not plotted since it is too noisy to visualize. SPSA is
implemented by replacing the metric tensor with an identity matrix.

![](../demonstrations/qnspsa/qnspsa_blocking.png){.align-center
width="80.0%"}

Efficiency improvement
======================

Let's do a deep dive on how to further improve the execution efficiency
of the code. In the code example above, we compute gradient, metric
tensor, and the loss values through individual calls on the
`QNode.__call__()` function (in this example, `cost_function()`). In a
handwavy argument, each `QNode.__call__()` does the following two
things: (1) it constructs a tape with the given parameters, and (2)
calls `qml.execute()` to execute the single tape.

However, in this use case, the better practice is to group the tapes and
call one `qml.execute()` on all the tapes. This practice utilizes the
batch execution feature from PennyLane, and has a few potential
advantages. Some simulators provide parallelization support, so that the
grouped tapes can be executed simutaneously. As an example, utilizing
the [task
batching](https://docs.aws.amazon.com/braket/latest/developerguide/braket-batching-tasks.html?tag=local002)
feature from the Braket SV1 simulator, we are able to reduce the
optimization time by 75% for large circuits. For quantum hardware,
sending tapes in batches could also enable further efficiency
improvement in circuit compilation.

With this rewriting, the complete optimizer class is provided in the
following cell.


Let's see how it performs on our QAOA example:


::: {.rst-class}
sphx-glr-script-out

``` {.none}
Step 0: cost = -2.0730
Step 40: cost = -2.5390
Step 80: cost = -2.7240
Step 120: cost = -2.6760
Step 160: cost = -2.7540
Step 200: cost = -2.7060
Step 240: cost = -2.8110
Step 280: cost = -2.7930
```
:::

The optimizer performs reasonably well: the loss drops over optimization
steps and converges finally. We then reproduce the benchmarking test
between the gradient descent, quantum natural gradient descent, SPSA and
QN-SPSA in Fig. 1(b) of reference with the following job. You can find a
more detailed version of the example in this
[notebook](https://github.com/aws/amazon-braket-examples/blob/main/examples/hybrid_jobs/6_QNSPSA_optimizer_with_embedded_simulator/qnspsa_with_embedded_simulator.ipynb),
with its dependencies in the [source\_scripts]{.title-ref}
[folder](https://github.com/aws/amazon-braket-examples/blob/main/examples/hybrid_jobs/6_QNSPSA_optimizer_with_embedded_simulator/source_scripts/).

::: {.note}
::: {.title}
Note
:::

In order for the remainder of this demo to work, you will need to have
done 3 things:

1.  Copied the [source\_scripts]{.title-ref} folder (linked above) to
    your working directory
2.  Authenticated with AWS locally
3.  Granted yourself the appropriate permissions as described in this
    [AWS Braket setup
    document](https://docs.aws.amazon.com/braket/latest/developerguide/braket-enable-overview.html)
:::


Visualizing the job results, we get the following plot. The results are
well aligned with the observations from Gacon et al.[^1]. The stepwise
optimization times for GD, QNG, SPSA and QN-SPSA are measured to be
0.43s, 0.75s, 0.03s and 0.20s. In this example, the average behavior of
SPSA matches the one from GD. QNG performs the best among the 4
candidates, but it requires the most circuit executions and shots per
step. In particular, for QPUs, this is a severe disadvantage of this
method. From the more shot-frugal options, QN-SPSA demonstrates the
fastest convergence and best final accuracy, making it a promising
candidate for variational algorithms.

![](../demonstrations/qnspsa/qnspsa_braket.png){.align-center
width="80.0%"}

To sum up, in this tutorial, we showed step-by-step how we can implement
the QN-SPSA optimizer with PennyLane, along with a few tricks to further
improve the optimizer's performance. We also demonstrated how one can
scale up the benchmarking experiments with Amazon Braket Hybrid Jobs.

References
==========

About the author
================

[^1]: Gacon, J., Zoufal, C., Carleo, G., & Woerner, S. (2021).
    *Simultaneous perturbation stochastic approximation of the quantum
    fisher information*. [Quantum, 5,
    567](https://quantum-journal.org/papers/q-2021-10-20-567/).
